In [58]:
import pandas as pd
from pathlib import Path
import calendar
import warnings

# Ignore specific warning by category
warnings.filterwarnings("ignore")

# Import raw fire data
file_path=Path("Resources/fire_data/mapdataall.csv")
df=pd.read_csv(file_path, encoding="utf-8")

df.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_latitude,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,40.774000,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,https://incidentupdate.fire.ca.gov/incidents/2...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,1.000000,Wildfire,8f61f461-552d-4538-b186-35ab030da416,https://incidentupdate.fire.ca.gov/incidents/2...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,36.602575,NaN,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://incidentupdate.fire.ca.gov/incidents/2...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,34.288877,NaN,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,https://incidentupdate.fire.ca.gov/incidents/2...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,37.116295,NaN,357ffc13-bef9-48eb-810f-c5de851972eb,https://incidentupdate.fire.ca.gov/incidents/2...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False


In [11]:
#gather list of original columns
#list(df.columns)

In [60]:
#create new data frame with select columns
fire_raw = df[['incident_name',
 'incident_administrative_unit',
 'incident_county',
 'incident_acres_burned',
 'incident_longitude',
 'incident_latitude',
 'incident_dateonly_extinguished',
 'incident_dateonly_created'
]]

#rename columns
fire_raw = fire_df.rename(columns={
    'incident_name': "Name",
     'incident_administrative_unit': "Admin unit",
     'incident_county': "County",
     'incident_acres_burned': "Acres burned" ,
     'incident_longitude': "Lon",
     'incident_latitude': "Lat",
     'incident_dateonly_extinguished': "Date extinguished",
     'incident_dateonly_created': "Date created"
})

#fire_incidents_df.head()
fire_raw.head()

,Name,Admin unit,County,Acres burned,Lon,Lat,Date extinguished,Date created
0,Bridge Fire,Shasta-Trinity National Forest,Shasta,37.0,-122.309000,40.774000,2018-01-09,2017-10-31
1,Pala Fire,CAL FIRE San Diego Unit,San Diego,122.0,1.000000,1.000000,2009-05-25,2009-05-24
2,River Fire,CAL FIRE San Bernardino Unit,Inyo,407.0,-118.016510,36.602575,2013-02-28,2013-02-24
3,Fawnskin Fire,San Bernardino National Forest,San Bernardino,30.0,-116.941311,34.288877,2013-04-22,2013-04-20
4,Gold Fire,CAL FIRE Madera-Mariposa-Merced Unit,Madera,274.0,-119.635004,37.116295,2013-05-01,2013-04-30


In [65]:
#drop rows where county and acres burned columns are null
fire_clean = fire_raw.dropna(subset = ['Acres burned','County', 'Date extinguished'])

#convert to datetime
fire_clean['Date extinguished'] = pd.to_datetime(fire_clean.loc[:,'Date extinguished'])
fire_clean['Date created'] = pd.to_datetime(fire_clean.loc[:,'Date created'])

#calculate duration
fire_clean['Duration (days)'] = fire_clean.loc[:,'Date extinguished'] - fire_clean.loc[:,'Date created']

#add years columns. Note dates are currently in datetime
fire_clean["Year extinguished"] = fire_clean.loc[:,"Date extinguished"].dt.year
fire_clean["Year"] = fire_clean.loc[:,"Date created"].dt.year

#add months column by name (& number?) do not need extinguished only created
fire_clean["Month created (num)"] = fire_clean["Date created"].dt.month
fire_clean["Month"] = fire_clean["Month created (num)"].apply(lambda x: calendar.month_name[x])

fire_clean

,Name,Admin unit,County,Acres burned,Lon,Lat,Date extinguished,Date created,Duration (days),Year extinguished,Year,Month created (num),Month
0,Bridge Fire,Shasta-Trinity National Forest,Shasta,37.0,-122.309000,40.774000,2018-01-09,2017-10-31,70 days,2018,2017,10,October
1,Pala Fire,CAL FIRE San Diego Unit,San Diego,122.0,1.000000,1.000000,2009-05-25,2009-05-24,1 days,2009,2009,5,May
2,River Fire,CAL FIRE San Bernardino Unit,Inyo,407.0,-118.016510,36.602575,2013-02-28,2013-02-24,4 days,2013,2013,2,February
3,Fawnskin Fire,San Bernardino National Forest,San Bernardino,30.0,-116.941311,34.288877,2013-04-22,2013-04-20,2 days,2013,2013,4,April
4,Gold Fire,CAL FIRE Madera-Mariposa-Merced Unit,Madera,274.0,-119.635004,37.116295,2013-05-01,2013-04-30,1 days,2013,2013,4,April
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,South Fire,Ventura County Fire Department,Ventura,2715.0,-118.993800,34.327000,2023-12-12,2023-12-09,3 days,2023,2023,12,December
2193,Pine Fire,Kern County Fire Department,Kern,181.0,-118.869600,35.559300,2023-12-15,2023-12-15,0 days,2023,2023,12,December
2194,Border 42 Fire,CAL FIRE San Diego Unit,San Diego,50.0,-116.771800,32.603600,2023-12-22,2023-12-21,1 days,2023,2023,12,December
2195,Palmas Fire,Bureau of Land Management,Riverside,239.0,-115.827840,33.510379,2024-02-26,2024-02-20,6 days,2024,2024,2,February


In [66]:
fire_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1943 entries, 0 to 2196
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   Name                 1943 non-null   object         
 1   Admin unit           1934 non-null   object         
 2   County               1943 non-null   object         
 3   Acres burned         1943 non-null   float64        
 4   Lon                  1943 non-null   float64        
 5   Lat                  1943 non-null   float64        
 6   Date extinguished    1943 non-null   datetime64[ns] 
 7   Date created         1943 non-null   datetime64[ns] 
 8   Duration (days)      1943 non-null   timedelta64[ns]
 9   Year extinguished    1943 non-null   int32          
 10  Year                 1943 non-null   int32          
 11  Month created (num)  1943 non-null   int32          
 12  Month                1943 non-null   object         
dtypes: datetime64[ns](2), f

In [ ]:
# fire_incidents.to_csv("Resources/fire_data/fire_all_clean.csv", index_label="City_ID")

In [73]:
#THERE ARE MULTIPLE FIRES IN YEAR/MONTH/COUNTY. 
#Therefore you need to group by month and summarize the data per month.

#select columns
fires_df = fire_clean[["Name","Year", "Month","Month created (num)", "County", "Acres burned", "Duration (days)", "Lat", "Lon"]]

#drop "days" and convert to int
################# BETTER WAY?
fires_df["Duration (days)"] = fires_df["Duration (days)"].astype(str)
fires_df.loc[:, "Duration (days)"] = pd.to_numeric(fires_df["Duration (days)"].str.extract('(\d+)')[0], errors="coerce")
fires_df["Duration (days)"] = fires_df["Duration (days)"].astype(float)

fires_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1943 entries, 0 to 2196
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 1943 non-null   object 
 1   Year                 1943 non-null   int32  
 2   Month                1943 non-null   object 
 3   Month created (num)  1943 non-null   int32  
 4   County               1943 non-null   object 
 5   Acres burned         1943 non-null   float64
 6   Duration (days)      1943 non-null   float64
 7   Lat                  1943 non-null   float64
 8   Lon                  1943 non-null   float64
dtypes: float64(4), int32(2), object(3)
memory usage: 136.6+ KB


In [78]:
fire_duplicate_df = fires_df.loc[fires_df['County'].str.contains(','), :]
fire_duplicate_df.head()

,Name,Year,Month,Month created (num),County,Acres burned,Duration (days),Lat,Lon
13,Grand Fire,2013,May,5,"Kern, Ventura",4346.0,6.0,34.78610,-118.94110
33,Sonoma-Lake-Napa Unit Lightning Fires,2013,June,6,"Sonoma, Solano, Lake, Colusa",28.0,4.0,38.42290,-122.04370
82,50 Fire,2013,July,7,"El Dorado, Sacramento",163.0,0.0,38.63987,-121.08438
173,Butts Fire,2014,July,7,"Napa, Lake",4300.0,8.0,38.66400,-122.45400
188,Sand Fire,2014,July,7,"Amador, El Dorado",4240.0,8.0,38.56910,-120.78190


In [80]:
######### ReLOOK TO SIMPLIFY IF TIME

def expand_counties(row):
    counties = row['County'].split(", ")
    num_counties = len(counties)
    # Create new row for each county with the duration unchanged
    return [
        {**row, 'County': county, 'Acres burned': row['Acres burned'] / num_counties, 'Duration (days)': row['Duration (days)']}
        for county in counties
    ]

# Apply the function to each row and expand into a list of dictionaries
expanded_rows = [new_row for index, row in fire_duplicate_df.iterrows() for new_row in expand_counties(row)]

# Convert the list of dictionaries to a DataFrame
split_county_df = pd.DataFrame(expanded_rows)

# Preview
split_county_df

,Name,Year,Month,Month created (num),County,Acres burned,Duration (days),Lat,Lon
0,Grand Fire,2013,May,5,Kern,2173.0,6.0,34.786100,-118.941100
1,Grand Fire,2013,May,5,Ventura,2173.0,6.0,34.786100,-118.941100
2,Sonoma-Lake-Napa Unit Lightning Fires,2013,June,6,Sonoma,7.0,4.0,38.422900,-122.043700
3,Sonoma-Lake-Napa Unit Lightning Fires,2013,June,6,Solano,7.0,4.0,38.422900,-122.043700
4,Sonoma-Lake-Napa Unit Lightning Fires,2013,June,6,Lake,7.0,4.0,38.422900,-122.043700
...,...,...,...,...,...,...,...,...,...
94,Fremont Fire,2021,September,9,Sonoma,58.0,1.0,38.269863,-122.416983
95,Mosquito Fire,2022,September,9,El Dorado,38394.0,46.0,39.005910,-120.744700
96,Mosquito Fire,2022,September,9,Placer,38394.0,46.0,39.005910,-120.744700
97,Vineyard Fire,2023,July,7,Monterey,196.0,1.0,35.800000,-120.816900


In [98]:
#set of counties to drop
counties_to_drop = set(fire_duplicate_df['County'])

#rows where the 'County' is not in the counties_to_drop
mask = ~fires_df['County'].isin(counties_to_drop)


#apply the mask to the df to filter out unwanted rows
fire_data = fires_df[mask]

## confirmed original number of rows 1943 - 39 duplicate rows is 1904

#add rows
fires_all_ungrouped = pd.concat([fire_data, split_county_df], axis=0, ignore_index=True)
fires_all_ungrouped

# Export clean fires df for mapping before removing Lat and Long
fires_all_ungrouped.to_csv("Resources/fire_data/fires_all_ungrouped.csv", index_label="City_ID")

#99+1904 =2003

In [100]:
#groupby
fire_grouped=fires_all_df.groupby(['Year','Month','Month created (num)', 'County'])

#calc acres avg
acres_burned_avg = fire_grouped["Acres burned"].mean()

#calc acres total
acres_burned_tot = fire_grouped["Acres burned"].sum()

#calc rounded avg days
duration_days_avg = fire_grouped["Duration (days)"].mean()

#sum count of incidents
tot_fire_incidents = fire_grouped["Acres burned"].count()

summary_fire = pd.DataFrame({
    "Acres Burned Avg": acres_burned_avg,
    "Acres Burned Tot": acres_burned_tot,
    "Fire Duration Avg (days)": duration_days_avg,
    "Incident Count Tot": tot_fire_incidents
 }).reset_index()

summary_fire
#drop 2024 and 1969
fire_all_grouped = summary_fire.loc[(summary_fire["Year"]>=2013) & (summary_fire["Year"]<2024)]
fire_all_grouped

fire_all_grouped.to_csv("Resources/fire_data/fire_all_grouped.csv", index_label="City_ID")

In [54]:
# #county key metrics

# #groupby county
# county_group = fire_incidents.groupby(["Month","County"]) #ADD GROUP BY MONTH FIRST

# #create summary data frame per county
# county_summary = county_group.agg(
#     #count incidents
#     Incident_count=("County","count"), 
    
#     #total acre burned
#     Total_acres_burned=("Acres burned", "sum"),
    
#     #avg acres burned and round to whole number
#     Avg_acres_burned=("Acres burned", "mean"),
    
#     #avg duration and only exctract days as an integer  
#     Avg_duration=("Duration (days)", lambda x: x.mean().days)
# )

# # convert 'total_acres_burned' and 'avg_acres_burned' to whole numbers/integer
# county_summary['Total_acres_burned'] = county_summary['Total_acres_burned'].astype(int)
# county_summary['Avg_acres_burned'] = county_summary['Avg_acres_burned'].round(0).astype(int)

# #sort by descending incident count
# #county_summary = county_summary.sort_values(by=["incident_count"], ascending=False)

# #preview
# county_summary.head(25)

# #county_summary.info()
# county_summary = county_summary.reset_index()
# county_summary

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Month               410 non-null    object
 1   County              410 non-null    object
 2   Incident_count      410 non-null    int64 
 3   Total_acres_burned  410 non-null    int64 
 4   Avg_acres_burned    410 non-null    int64 
 5   Avg_duration        410 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 19.3+ KB


In [18]:
# # year key metrics

# #groupby year created
# year_group = fire_incidents.groupby(["Year"])

# #create data frame with incident count and acres burned
# year_summary = year_group.agg(
#     Incident_count=("County","count"), 
#     Total_acres_burned=("Acres burned", "sum")
# )

# #sort by descending incident count
# year_created_summary = year_summary.sort_values(by=["Year"], ascending=False)

# #preview
# year_created_summary.head(25)

,Incident_count,Total_acres_burned
Year,,
2024,2,250.0
2023,93,310026.0
2022,84,140415.0
2021,157,2289096.0
2020,205,2453742.0
2019,205,200634.0
2018,300,1529897.0
2017,427,1258294.0
2016,155,452101.0


In [19]:
# #month key metrics

# #groupby month created 
# month_group = fire_incidents.groupby(["Month"])

# #create data frame with incident count and acres burned
# month_summary = month_group.agg(
#     Incident_count=("County","count"),
#     #incident_avg=("county", "mean"),
#     Total_acres_burned=("Acres burned", "sum"),
#     Avg_acres_burned=("Acres burned", "mean"),
#     County_count=("County", "nunique")
# )

# #sort by descending incident count
# month_summary = month_summary.sort_values(by=["Incident_count"], ascending=False)

# #preview
# month_summary.head(25)

,Incident_count,Total_acres_burned,Avg_acres_burned,County_count
Month,,,,
July,478,3749362.0,7843.853556,66
June,427,412379.0,965.758782,56
August,354,3852007.0,10881.375706,61
September,220,668800.0,3040.000000,54
May,165,125753.0,762.139394,39
October,150,408635.0,2724.233333,46
April,44,14373.0,326.659091,24
November,43,272865.0,6345.697674,24
December,27,321923.0,11923.074074,14
